In [28]:
from pathlib import Path
import sys  
import os
import pandas as pd 

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [21]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

In [20]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from scipy.sparse import hstack, vstack
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [11]:
URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [12]:
ICM_all.T

<COOrdinate sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (94331, 38121)>

In [6]:
ICM_all

<COOrdinate sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

In [14]:
combined_data = vstack((URM_all, ICM_all.T))

In [15]:
combined_data

<COOrdinate sparse matrix of dtype 'float64'
	with 4704647 stored elements and shape (130067, 38121)>

In [16]:
combined_data_train, combined_data_validation = split_train_in_two_percentage_global_sample(combined_data, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(combined_data_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 55646 (42.8%) Users that have less than 1 test interactions


In [18]:
MAP = []

In [22]:

combined_data_train_coo = combined_data_train.tocoo()


for topK_ in [100, 200, 300]:
    for l1_ratio_ in np.linspace(0.001, 1, 10):
        for alpha_ in np.linspace(0.00001, 0.0005, 3):
            slim_model = SLIMElasticNetRecommender(combined_data_train_coo, verbose=True)
            slim_model.fit(l1_ratio=l1_ratio_, alpha = alpha_, topK = topK_)

            result_dict, _ = evaluator_validation.evaluateRecommender(slim_model)
            MAP.append(('topK={}'.format(topK_), 'l1_ratio={}'.format(l1_ratio_), 'alpha={}'.format(alpha_), result_dict["MAP"][10]))

            print(MAP[-1])
            

SLIMElasticNetRecommender: URM Detected 11551 ( 8.9%) users with no interactions.
SLIMElasticNetRecommender: Processed 2037 ( 5.3%) in 5.00 min. Items per second: 6.78
SLIMElasticNetRecommender: Processed 4119 (10.8%) in 10.00 min. Items per second: 6.86
SLIMElasticNetRecommender: Processed 4855 (12.7%) in 15.22 min. Items per second: 5.32
SLIMElasticNetRecommender: Processed 5271 (13.8%) in 24.43 min. Items per second: 3.60
SLIMElasticNetRecommender: Processed 5394 (14.1%) in 33.53 min. Items per second: 2.68
SLIMElasticNetRecommender: Processed 5542 (14.5%) in 42.63 min. Items per second: 2.17
SLIMElasticNetRecommender: Processed 5692 (14.9%) in 51.73 min. Items per second: 1.83
SLIMElasticNetRecommender: Processed 5846 (15.3%) in 1.01 hour. Items per second: 1.60
SLIMElasticNetRecommender: Processed 5996 (15.7%) in 1.17 hour. Items per second: 1.43
SLIMElasticNetRecommender: Processed 6138 (16.1%) in 1.32 hour. Items per second: 1.29
SLIMElasticNetRecommender: Processed 6294 (16.5%)

In [24]:
max_tuple = max(MAP, key=itemgetter(3))
print('The best combination for the SSLIM recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))

The best combination for the SSLIM recommender is (topK=200,l1_ratio=0.001,alpha=0.000255), with a MAP = 0.05361971776612382


In [25]:
slim_model = SLIMElasticNetRecommender(combined_data, verbose=True)
slim_model.fit(l1_ratio=0.001, alpha = 0.000255, topK = 200)

SLIMElasticNetRecommender: Processed 1854 ( 4.9%) in 5.00 min. Items per second: 6.18
SLIMElasticNetRecommender: Processed 3733 ( 9.8%) in 10.00 min. Items per second: 6.22
SLIMElasticNetRecommender: Processed 5594 (14.7%) in 15.00 min. Items per second: 6.21
SLIMElasticNetRecommender: Processed 7424 (19.5%) in 20.01 min. Items per second: 6.18
SLIMElasticNetRecommender: Processed 9327 (24.5%) in 25.01 min. Items per second: 6.22
SLIMElasticNetRecommender: Processed 11241 (29.5%) in 30.01 min. Items per second: 6.24
SLIMElasticNetRecommender: Processed 13144 (34.5%) in 35.01 min. Items per second: 6.26
SLIMElasticNetRecommender: Processed 15063 (39.5%) in 40.01 min. Items per second: 6.27
SLIMElasticNetRecommender: Processed 16948 (44.5%) in 45.01 min. Items per second: 6.27
SLIMElasticNetRecommender: Processed 18749 (49.2%) in 50.02 min. Items per second: 6.25
SLIMElasticNetRecommender: Processed 20641 (54.1%) in 55.02 min. Items per second: 6.25
SLIMElasticNetRecommender: Processed 2

In [26]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [29]:
write_submission_lib_model(slim_model, filename='sslim_urm_icm_200.csv')